In [27]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/CycPeptMPDB_Peptide_Assay_Caco2 (1).csv')
df.shape

(1332, 247)

In [29]:
last_id = df['ID'].max()
print('last_id: ', last_id)

last_id:  8456


In [30]:
#Step 1: Removing unreliable datapoints (datapoints with Permeability value <= -10) and selecting the required Columns
df = df[df['Permeability'] > -10]
selected_columns = ['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']
new_df = df[selected_columns]
new_df.shape

(1311, 5)

In [31]:
new_df.head(10)

,ID,SMILES,Permeability,Sequence,MolWt
0,32,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-5.31,"['L', 'dP', 'meY', 'L', 'Me_dL', 'meL']",755.014
1,33,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-5.81,"['L', 'dP', 'meY', 'L', 'Me_dL', 'meS']",728.932
2,40,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-6.34,"['L', 'dP', 'meY', 'L', 'Me_dL', 'meD']",756.942
3,41,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-6.56,"['L', 'dP', 'meY', 'L', 'Me_dL', 'meK']",770.029
5,927,CC(C)C[C@@H]1NC(=O)CNC(=O)[C@@H]2CCCN2[C@H](C(...,-6.40,"['Mono13', 'G', 'L', 'G', 'F']",627.787
6,982,CC[C@H](C)[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@H](...,-5.28,"['Mono14', 'A', 'F', 'P', 'I', 'P']",721.925
7,983,CC[C@H](C)[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@H](...,-5.25,"['Mono30', 'A', 'F', 'P', 'I', 'P']",721.925
8,984,CC[C@@H](C)[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@H]...,-5.28,"['Mono14', 'A', 'F', 'P', 'L', 'P']",721.925
9,985,CC[C@H](C)[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@H](...,-5.32,"['Mono31', 'A', 'F', 'P', 'I', 'P']",707.898
10,986,CC[C@@H](C)[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@H]...,-5.31,"['Mono14', 'A', 'F', 'P', 'V', 'P']",707.898


In [34]:
#Step 2: curating duplicate dapaoints based on SMILES representation
duplicates = new_df[new_df.duplicated(subset='SMILES', keep=False)]
non_duplicates = new_df[~new_df.duplicated(subset='SMILES', keep=False)]
print("DataFrame with duplicates:")
print(duplicates)
print("\nDataFrame without duplicates:")
print(non_duplicates)
duplicates.to_csv('/home/users/akshay/PCPpred/Caco2/data/duplicates_Caco2.csv', index=False)

DataFrame with duplicates:
        ID                                             SMILES  Permeability  \
0       32  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...     -5.310000   
23     999  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...     -4.750000   
49    1027  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...     -4.400000   
55    1049  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...     -5.910000   
62    1056  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...     -6.240000   
...    ...                                                ...           ...   
1253  8080  CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)C[C@@H...     -7.000000   
1256  8083  CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)C[C@@H...     -7.096910   
1308  8369  CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)C[C@@H...     -7.096910   
1310  8371  CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)C[C@@H...     -6.958607   
1315  8376  CC[C@H](C)[C@@H]1NC(=O)[C@H](C)N(C)C(=O)C[C@@H...     -7.000000   

                        

In [35]:
duplicates = new_df[new_df.duplicated(subset='SMILES', keep=False)]

non_duplicates = new_df[~new_df.duplicated(subset='SMILES', keep=False)]
#Permeability and MolWt of duplicate SMILES were averaged, and Most abundandant sequence was chosen
grouped_duplicates = duplicates.groupby('SMILES').agg(
    Permeability=('Permeability', 'mean'),   
    MolWt=('MolWt', 'mean'),   
    Sequence=('Sequence', lambda x: x.mode()[0]), 
).reset_index()

#Unique ID's were given to these curated datapoints, where ID's start from the last documented id in CycPeptMPDB_Peptide_Assay_PAMPA (5)
grouped_duplicates['ID'] = range(last_id + 1, last_id + 1 + len(grouped_duplicates))

grouped_duplicates = grouped_duplicates[['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']]

non_duplicates_ordered = non_duplicates[['ID', 'SMILES', 'Permeability', 'Sequence', 'MolWt']]
merged_df = pd.concat([non_duplicates_ordered, grouped_duplicates], ignore_index=True)

print("DataFrame with processed duplicates:")
print(grouped_duplicates)

print("\nDataFrame without duplicates:")
print(non_duplicates)

print("Merged DataFrame:")
print(merged_df)

grouped_duplicates.to_csv('/home/users/akshay/PCPpred/Caco2/data/grouped_duplicates_Caco2.csv', index=False)
non_duplicates.to_csv('/home/users/akshay/PCPpred/Caco2/data/non_duplicates_Caco2.csv', index=False)
merged_df.to_csv('/home/users/akshay/PCPpred/Caco2/data/CycPeptMPDB_data_Main_Caco2.csv', index=False)

DataFrame with processed duplicates:
      ID                                             SMILES  Permeability  \
0   8457  CC(C)CCOC[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)CN...     -5.720000   
1   8458  CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)C[C@@H](C(...     -5.700000   
2   8459  CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)C[C@@H](C(...     -5.800000   
3   8460  CC(C)C[C@@H]1NC(=O)[C@H](C)N(C)C(=O)C[C@@H](C(...     -5.850000   
4   8461  CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[C@H]...     -5.850000   
5   8462  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...     -5.175000   
6   8463  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C...     -5.755000   
7   8464  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...     -5.870000   
8   8465  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...     -5.700000   
9   8466  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...     -5.770000   
10  8467  CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...     -5.960000   
11  8468  CC(C)C[C@@H]1NC(=O)[C@H](Cc2c

In [36]:
#Step 3: Splitting the Main dataset into Train and Test in ration 0.8:0.2
df = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/CycPeptMPDB_data_Main_Caco2.csv')
df

,ID,SMILES,Permeability,Sequence,MolWt
0,33,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-5.810,"['L', 'dP', 'meY', 'L', 'Me_dL', 'meS']",728.932
1,40,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-6.340,"['L', 'dP', 'meY', 'L', 'Me_dL', 'meD']",756.942
2,41,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccc(O)cc2)N(C)C(=O...,-6.560,"['L', 'dP', 'meY', 'L', 'Me_dL', 'meK']",770.029
3,927,CC(C)C[C@@H]1NC(=O)CNC(=O)[C@@H]2CCCN2[C@H](C(...,-6.400,"['Mono13', 'G', 'L', 'G', 'F']",627.787
4,982,CC[C@H](C)[C@@H]1NC(=O)[C@@H]2CCCN2C(=O)[C@H](...,-5.280,"['Mono14', 'A', 'F', 'P', 'I', 'P']",721.925
...,...,...,...,...,...
1255,8497,CC[C@H](C)[C@H]1C(=O)N[C@@H]([C@@H](C)O)C(=O)N...,-5.740,"['meA', 'V', 'meL', 'meF', 'Ser(Ph(3-Cl))', 'm...",1398.155
1256,8498,CC[C@H](C)[C@H]1C(=O)N[C@@H]([C@@H](C)O)C(=O)N...,-5.960,"['meV', 'Mono49', 'meL', 'meF', 'L', 'meF', 'm...",1480.877
1257,8499,CC[C@H](C)[C@H]1C(=O)N[C@@H]([C@@H](C)O)C(=O)N...,-5.595,"['meA', 'Nle(CHF2)', 'meL', 'meF', 'V', 'meF',...",1405.782
1258,8500,CC[C@H](C)[C@H]1C(=O)N[C@@H]([C@@H](C)O)C(=O)N...,-5.890,"['meA', 'Ser(Ph(3-Cl))', 'meL', 'meF', 'A', 'm...",1412.182


In [37]:
#Main dataset is sorted based on Molecular weight in descending order
df_main = df.sort_values(by='MolWt', ascending= False)
df_main

,ID,SMILES,Permeability,Sequence,MolWt
258,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,"['Me_Bal', 'meF', 'meL', 'meA', 'meF', 'I', 'm...",1664.156
259,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,"['meA', 'meF', 'I', 'meA', 'meL', 'meL', 'T', ...",1664.156
261,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,"['dA', 'T', 'meF', 'I', 'meA', 'meL', 'meF', '...",1650.129
133,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,"['dL', 'Me_Phe(3-Cl)', 'L', 'meL', 'T', 'Bu_Gl...",1587.863
225,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,"['Me_dA', 'meA', 'meF', 'T', 'meA', 'meL', 'I'...",1574.031
...,...,...,...,...,...
482,5603,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)CCCCNC(=O)CN(...,-7.18,"['A', 'L', 'Bn_Gly', '5-Ava']",430.549
445,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,"['A', 'L', 'Mono77']",416.566
444,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,"['A', 'L', 'Mono76']",402.539
443,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,"['A', 'L', 'Mono75']",388.512


In [38]:
df_main['MolWt']

258    1664.156
259    1664.156
261    1650.129
133    1587.863
225    1574.031
         ...   
482     430.549
445     416.566
444     402.539
443     388.512
442     374.485
Name: MolWt, Length: 1260, dtype: float64

In [39]:
#Every fifth datapoint is allocated to test set and remaining datapoints to Train set
test_mask = np.arange(len(df_main)) % 5 == 0
test_df = df_main[test_mask]
test_df.shape

(252, 5)

In [40]:
train_df = df_main[~test_mask]
train_df.shape

(1008, 5)

In [41]:
print(train_df["Permeability"].max())
print(train_df["Permeability"].min())
print(train_df.shape)

-4.3
-8.4
(1008, 5)


In [42]:
print(test_df["Permeability"].max())
print(test_df["Permeability"].min())
print(test_df.shape)

-4.2
-8.2
(252, 5)


In [43]:
test_df.to_csv("/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv",index=False)
train_df.to_csv("/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv",index=False)

In [44]:
train_df

,ID,SMILES,Permeability,Sequence,MolWt
259,2065,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.22,"['meA', 'meF', 'I', 'meA', 'meL', 'meL', 'T', ...",1664.156
261,2067,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.24,"['dA', 'T', 'meF', 'I', 'meA', 'meL', 'meF', '...",1650.129
133,1914,CCCCN1CC(=O)N(C)[C@@H](CC(C)C)C(=O)N[C@@H](CCC...,-8.00,"['dL', 'Me_Phe(3-Cl)', 'L', 'meL', 'T', 'Bu_Gl...",1587.863
225,2026,CC[C@H](C)[C@@H]1NC(=O)[C@H](CC(C)C)N(C)C(=O)[...,-5.64,"['Me_dA', 'meA', 'meF', 'T', 'meA', 'meL', 'I'...",1574.031
139,1920,CCCCN1CC(=O)N(C)[C@@H](Cc2cccc(Cl)c2)C(=O)N[C@...,-7.05,"['Bu_Gly', 'Me_Phe(3-Cl)', 'Gln(Mes)', 'meL', ...",1567.445
...,...,...,...,...,...
483,5604,CC(C)CN1CC(=O)N[C@@H](Cc2ccccc2)C(=O)NCCCCC(=O...,-6.38,"['A', 'iBu_Gly', 'F', '5-Ava']",430.549
445,2470,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.60,"['A', 'L', 'Mono77']",416.566
444,2469,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.70,"['A', 'L', 'Mono76']",402.539
443,2468,CC(C)C[C@@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2cccc...,-4.90,"['A', 'L', 'Mono75']",388.512


In [45]:
test_df

,ID,SMILES,Permeability,Sequence,MolWt
258,2064,CC[C@H](C)[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=...,-7.19,"['Me_Bal', 'meF', 'meL', 'meA', 'meF', 'I', 'm...",1664.156
1128,8066,CC[C@H]1C(=O)N[C@@H](COCCC(C)C)C(=O)N(C)[C@@H]...,-6.21,"['Me_dA', 'Me_Abu', 'Ser(isoamyl)', 'meL', 'Se...",1572.049
262,2068,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)N(C)C(=O)[C...,-7.24,"['dL', 'meL', 'meF', 'meF', 'Ser(tBu)', 'meF',...",1552.024
397,2234,CC(C)C[C@H]1C(=O)N[C@@H](COC(C)(C)C)C(=O)N(C)[...,-5.85,"['Mono60', 'Me_Phe(3-Cl)', 'V', 'meV', 'T', 'S...",1537.344
393,2230,CC(C)C[C@H]1C(=O)N[C@H](C(=O)N2CCCCC2)CC(=O)N[...,-5.96,"['dA', 'Mono59', 'Ser(tBu)', 'meL', 'meL', 'me...",1530.953
...,...,...,...,...,...
1206,8448,CC(C)C[C@@H]1NC(=O)CN(C)C(=O)[C@H](Cc2ccccc2)N...,-5.88,"['Sar', 'L', 'Mono122']",440.544
453,2478,CC(C)C[C@@H]1NC(=O)[C@H](C)NCCCCCCNC(=O)[C@H](...,-4.50,"['L', 'F', 'Mono84']",430.593
452,2477,CC(C)C[C@@H]1NC(=O)[C@H](Cc2ccccc2)NC(=O)[C@H]...,-4.20,"['F', 'L', 'Mono84']",430.593
456,2481,CC(C)C[C@H]1NC(=O)[C@H](C)NC(=O)[C@H](Cc2ccccc...,-4.50,"['A', 'dL', 'Mono78']",430.593


In [47]:
import pandas as pd

df = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Train_Caco2.csv')

with open('/home/users/akshay/PCPpred/Caco2/data/Train_caco2.smi', 'w') as f:
    for _, row in df.iterrows():
        smiles = row['SMILES']
        id = row['ID']
        f.write(f"{smiles} {id}\n")
print("train.smi file has been created successfully.")

train.smi file has been created successfully.


In [48]:
df = pd.read_csv('/home/users/akshay/PCPpred/Caco2/data/Test_Caco2.csv')

with open('/home/users/akshay/PCPpred/Caco2/data/Test_caco2.smi', 'w') as f:
    for _, row in df.iterrows():
        smiles = row['SMILES']
        id = row['ID']
        f.write(f"{smiles} {id}\n")
print("test.smi file has been created successfully.")

test.smi file has been created successfully.
